# Predicting Hospital Readmission for Diabetes Patients

## Objective
Demonstrate an end-to-end machine learning pipeline to predict whether a patient will be readmitted to a hospital after a diabetes diagnosis using data maintained by UCI detailing 10 years of clinical care at U.S. hospitals and integrated delivery networks.

## Table of Contents
* [Data Loading](#data_loading)
* [Data Cleaning & Preprocessing](#data_cleaning)
* [Exploratory Data Analysis (EDA)](#eda)
    * [Expore numeric data](#eda_numeric)
    * [Explore categorical data](#eda_categorical)
* [Feature Engineering](#feature_engineering)
* [Model Training & Evaluation](#model_training)

In [1]:
import io
import zipfile

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

import requests

%matplotlib inline

## Data Loading <a class="anchor" id="data_loading"></a>

In [2]:
# URL of the dataset
url = "https://archive.ics.uci.edu/static/public/296/diabetes+130-us+hospitals+for+years+1999-2008.zip"

try:
    print("Downloading dataset...")
    response = requests.get(url, stream=True)
    response.raise_for_status() # raises error for bad HTTP responses

    print("Download successful! Extracting files...")
    zip_file = zipfile.ZipFile(io.BytesIO(response.content)) # open ZIP in memory

    # list the files
    files_list = zip_file.namelist()
    print("Files contained in ZIP:\n", ", ".join(files_list))

    # create a dictionary to store the files
    files = {}

    for file_name in files_list:
        with zip_file.open(file_name) as f:
            files[file_name] = pd.read_csv(f)

    # assign the DataFrames to variables
    df_diabetes = files.get("diabetic_data.csv")
    df_ids = files.get("IDS_mapping.csv")

except requests.exceptions.RequestException as e:
    print(f"Error downloading dataset: {e}")

    # TODO: Possibly comment out the next three lines
    print("Retrieving from local directory...")
    df_diabetes = pd.read_csv("~/Downloads/diabetes+130-us+hospitals+for+years+1999-2008/diabetic_data.csv")
    df_ids = pd.read_csv("~/Downloads/diabetes+130-us+hospitals+for+years+1999-2008/IDS_mapping.csv")

except zipfile.BadZipFile:
    print("Error processing ZIP file")

except OSError as e:
    print(f"Error opening file: {e}")

except KeyError as e:
    print(f"Error retrieving file: {e}")

Download successful! Extracting files...
Files contained in ZIP:
 diabetic_data.csv, IDS_mapping.csv


## Data Cleaning & Preprocessing <a class="anchor" id="data_cleaning"></a>

In [3]:
# prepare the IDs DF for merging
df_ids.dropna(inplace=True)
df_filtered_ids = df_ids[df_ids["admission_type_id"].str.isdigit()].copy()
df_grouped_ids = df_filtered_ids.groupby("admission_type_id")["description"].apply(lambda x: " | ".join(x)).reset_index()

# prepare the diabetes DF for merging
df_diabetes["admission_type_id"] = df_diabetes["admission_type_id"].astype(str)

# merge the DFs
df = df_diabetes.merge(df_grouped_ids, how="left", on="admission_type_id")

In [4]:
# look at the data types
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id           object
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

In [ ]:
# TODO: cast any identifer columns to an object type
# cols_to_convert = df.filter(regex=r"_(?:id|nbr)$").columns
# df[cols_to_convert] = df[cols_to_convert].astype(str)

In [6]:
# identify columns using a question mark as a null value
question_mark_cols = ", ".join(df.columns[(df == "?").any()])
print("Columns using \"?\" to signify null values:\n", question_mark_cols)

# replace "?" rows with NaN
df.where(df != "?", np.nan, inplace=True)

Columns using "?" to signify null values:
 race, weight, payer_code, medical_specialty, diag_1, diag_2, diag_3


In [7]:
def display_missing_value_columns(df):
    """Generates a DataFrame analyzing columns' null values.

    Args:
        df (pd.DataFrame): A dataset.

    Returns:
        pd.DataFrame: A DataFrame containing null statistics.
    """

    # get columns and their missing value counts
    missing_values = df.isna().sum()
    missing_values = missing_values[missing_values > 0].reset_index()

    # rename the columns and calculate the percentage of nulls
    missing_values.columns = ["column_name", "null_count"]
    missing_values["null_perc"] = missing_values["null_count"] / df.shape[0]

    # sort the values
    missing_values.sort_values(by="null_perc", ascending=False, inplace=True)

    return missing_values

In [8]:
# create a DF showing missing value statistics
display_missing_value_columns(df)

,column_name,null_count,null_perc
1,weight,98569,0.968585
7,max_glu_serum,96420,0.947468
8,A1Cresult,84748,0.832773
3,medical_specialty,49949,0.490822
2,payer_code,40256,0.395574
0,race,2273,0.022336
6,diag_3,1423,0.013983
5,diag_2,358,0.003518
4,diag_1,21,0.000206


In [ ]:
# drop columns with 30% or more values missing
non_na_vals = df.shape[0] - (0.3 * df.shape[0])
df.dropna(thresh=non_na_vals, axis=1, inplace=True)

# drop the remaining null rows
df.dropna(inplace=True)

(Rows, cols): (101766, 51)
(Rows, cols): (101766, 46)
(Rows, cols): (98053, 46)


In [ ]:
# check for duplicates
print(f"Total duplicate rows: {df.duplicated().sum()}")
df[df.duplicated()]

In [25]:
# check for duplicate patient numbers (i.e., more than one encounter per patient)
grouped_patient_nbrs = df.groupby("patient_nbr")["encounter_id"].count().reset_index()
grouped_patient_nbrs.sort_values(by="encounter_id", ascending=False)

,patient_nbr,encounter_id
60765,88785891,39
60329,88227540,23
17092,23199021,23
11723,1660293,23
19122,23643405,22
...,...,...
9499,138575831,1
32757,41013234,1
9498,138558794,1
32759,41014314,1


In [ ]:
# sort to keep only the patient number for the most recent encounter
df.sort_values(by=["patient_nbr", "encounter_id"], ascending=[True, False], inplace=True)
df.drop_duplicates(subset="patient_nbr", keep="first", inplace=True)

In [ ]:
def map_icd9_to_category(code):
    """Maps ICD-9 codes to disease categories.

    Args:
        code: A string value representing an ICD-9 code.
    Returns:
        category: A string value representing a disease classification.
    """

    try:
        code = str(code).strip()
        if code.startswith("250"):
            category =  "Diabetes-Related"
        elif code.startswith("E"):
            category =  "Supplementary Classification of External Causes of Injury and Poisoning"
        elif code.startswith("V"):
            category =  "Supplementary Classification of Factors influencing Health Status and Contact with Health Services"

        code = int(float(code))
        if 1 <= code <= 139:
            category =  "Infectious and Parasitic Diseases"
        elif 140 <= code <= 239:
            category =  "Neoplasms"
        elif 240 <= code <= 279:
            category =  "Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders"
        elif 280 <= code <= 289:
            category =  "Diseases of the Blood and Blood-forming Organs"
        elif 290 <= code <= 319:
            category =  "Mental Disorders"
        elif 320 <= code <= 389:
            category =  "Diseases of the Nervous System and Sense Organs"
        elif 390 <= code <= 459:
            category =  "Diseases of the Circulatory System"
        elif 460 <= code <= 519:
            category =  "Diseases of the Respiratory System"
        elif 520 <= code <= 579:
            category =  "Diseases of the Digestive System"
        elif 580 <= code <= 629:
            category =  "Diseases of the Genitourinary System"
        elif 630 <= code <= 679:
            category =  "Complications of Pregnancy, Childbirth, and the Puerperium"
        elif 680 <= code <= 709:
            category =  "Diseases of the Skin and Subcutaneous Tissue"
        elif 710 <= code <= 739:
            category =  "Diseases of the Musculoskeletal System and Connective Tissue"
        elif 740 <= code <= 759:
            category =  "Congenital Anomalies"
        elif 760 <= code <= 779:
            category =  "Certain Conditions originating in the Perinatal Period"
        elif 780 <= code <= 799:
            category =  "Symptoms, Signs and Ill-defined Conditions"
        elif 800 <= code <= 999:
            category =  "Injury and Poisoning"
        else:
            category =  "Other"

    except:
        category =  "Other"

    return category

In [ ]:
# convert ICD-9 codes to disease categories
df[["disease_class_1", "disease_class_2", "disease_class_3"]] = df[["diag_1", "diag_2", "diag_3"]].apply(lambda col: col.map(map_icd9_to_category, na_action="ignore"))

## Exploratory Data Analysis <a class="anchor" id="eda"></a>

### Explore numeric data <a class="anchor" id="eda_numeric"></a>

In [ ]:
# look at descriptive statistics for numeric columns
df.describe()

In [ ]:
def plot_numeric_distributions(df):
    """Generates a grid of boxplots and histograms for numeric columns.

    Args:
        df (pd.DataFrame): A dataset with numeric data.
    """

    # identify numeric columns
    numeric_cols = sorted(df.select_dtypes(include="number").columns)

    # get the number of numeric columns
    num_cols = len(numeric_cols)

    # create a figure and set of subplots
    fig, axes = plt.subplots(num_cols, 2, figsize=(10, 5 * num_cols))

    # create a side-by-side boxplot and histogram for each row
    for i, col in enumerate(numeric_cols):
        sns.boxplot(data=df, x=col, color="mediumvioletred", ax=axes[i, 0])
        axes[i, 0].set_title(f"Boxplot of {col}")

        sns.histplot(data=df, x=col, color="lightcoral", ax=axes[i, 1])
        axes[i, 1].set_title(f"Histogram of {col}")

    # prevent overlap and display figure
    plt.tight_layout()
    plt.show()

In [ ]:
# analyze the distributions in the numeric data
plot_numeric_distributions(df)

In [ ]:
def display_skewness_columns(df):
    """Generates a DataFrame analyzing columns' skewness.

    Args:
        df (pd.DataFrame): A dataset containing numeric data.

    Returns:
        pd.DataFrame: A DataFrame containing skewness values.
    """

    # check for skewness and rename columns
    skewness = df.select_dtypes(include="number").skew().reset_index()
    skewness.columns = ["column_name", "value"]

    # categorize the skewness
    conditions = [skewness["value"].abs().between(0.5, 1), skewness["value"].abs() > 1]
    choices = ["Moderate", "High"]
    skewness["skew_category"] = np.select(conditions, choices, default="Symmetric")

    # sort the values
    skewness.sort_values(by="value", ascending=False, inplace=True)

    return skewness

In [ ]:
# create a DF showing skewness
display_skewness_columns(df)

In [ ]:
# analyze the strength of relationships among numeric columns
plt.figure(figsize=(10, 8))
corr_df = df.filter(regex=r"^(?!.*_(?:id|nbr)$)").corr(numeric_only=True)
sns.heatmap(corr_df, cmap="PuRd", annot=True, fmt=".2f")
plt.show()

### Explore categorical data <a class="anchor" id="eda_categorical"></a>

In [ ]:
# look at descriptive statistics for categorical columns
df.describe(include="object")

In [ ]:
def display_cardinality_columns(df):
    """Generates a DataFrame analyzing columns' cardinality.

    Args:
        df (pd.DataFrame): A dataset.

    Returns:
        pd.DataFrame: A DataFrame containing cardinality statistics.
    """

    # get columns and their number of unique values
    cardinal_values = df.select_dtypes(include=["category", "object"]).nunique().reset_index()

    # rename the columns and categorize the skewness
    cardinal_values.columns = ["column_name", "num_unique"]
    cardinal_values["cardinality"] = cardinal_values["num_unique"].apply(lambda x: "Low" if x < 15 else "Moderate" if 15 <= x < 50 else "High")

    # sort the values
    cardinal_values.sort_values(by="num_unique", ascending=False, inplace=True)

    return cardinal_values

In [ ]:
# create a DF showing cardinality
display_cardinality_columns(df)

In [ ]:
def plot_categorical_distributions(df, threshold=5):
    """Generates a grid of violin and count plots for categorical columns.

    Args:
        df (pd.DataFrame): A dataset with categorical data.
        threshold (int): The max number of unique values a column should have.
    """

    # identify categorical columns and remove ID columns
    unique_values = df.select_dtypes(include=["category", "object"]).nunique()
    categorical_cols = sorted(unique_values[unique_values.values <= threshold].index)

    # get the number of categorical columns
    num_cols = len(categorical_cols)

    # create a figure and set of subplots
    fig, axes = plt.subplots(num_cols, 2, figsize=(10, 5 * num_cols))

    # create a side-by-side violin plot and count plot for each row
    for i, col in enumerate(categorical_cols):

        sns.violinplot(data=df, x=col, color="mediumvioletred", ax=axes[i, 0])
        axes[i, 0].set_title(f"Boxplot of {col}")
        axes[i, 0].tick_params(axis="x", rotation=45)

        sns.countplot(data=df, x=col, color="lightcoral", ax=axes[i, 1])
        axes[i, 1].set_title(f"Histogram of {col}")
        axes[i, 1].tick_params(axis="x", rotation=45)

    # prevent overlap and display figure
    plt.tight_layout()
    plt.show()

In [ ]:
# analyze the distributions in the categorical data
plot_categorical_distributions(df)

In [ ]:
def display_chi_square_columns(df, target_col):
    """Generates a DataFrame analyzing columns' null values.

    Args:
        df (pd.DataFrame): A dataset containing categorical columns.
        target_col (str): A variable that is being predicted on.

    Returns:
        pd.DataFrame: A DataFrame containing chi-square and p-value statistics.
    """

    # get categorical columns, except for identifier columns and the target
    categorical_cols = df.select_dtypes(include=["category", "object"]).columns
    categorical_df = df[categorical_cols].copy()
    categorical_df.drop(columns=["encounter_id", "patient_nbr", target_col])

    # initialize a dictionary to hold the chi-square test results
    result_dict = {"column_name": [], "chi_sq": [], "p_val": []}

    for col in categorical_df:

        # compute the contingency table
        contingency_table = pd.crosstab(df[col], df[target_col])

        # computes the chi-square statistic and p-value
        chi_sq, p_val, _, _ = chi2_contingency(contingency_table)

        # append the results
        result_dict["column_name"].append(col)
        result_dict["chi_sq"].append(round(chi_sq, 2))
        result_dict["p_val"].append(round(p_val, 2))

    # create a DataFrame from the results
    result_df = pd.DataFrame(result_dict).sort_values(by="p_val")
    result_df["is_significant"] = result_df["p_val"] < 0.05

    print(f"Columns with/without a significant association to the column, {target_col}:")

    return result_df

In [ ]:
display_chi_square_columns(df, "readmitted")

## Feature Engineering <a class="anchor" id="feature_engineering"></a>